# Cartographie des Musées de France : Une Analyse Géospatiale


Le présent notebook explore les données relatives aux musées de France, disponibles sur le site data.culture.gouv.fr. Le jeu de données utilisé provient de l'URL suivante : https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin. Ce dataset fournit des informations détaillées sur les musées, notamment leur localisation géographique, leur nom officiel, leur adresse, ainsi que d'autres informations pertinentes.


L'objectif principal de cette analyse est de préparer les données pour créer une carte représentant les musées regroupés selon un buffer carré de 1 km. Cette visualisation permettra d'identifier les zones géographiques avec une forte concentration de musées, offrant ainsi une perspective intéressante sur la répartition de ces institutions culturelles en France.


## Méthodologie


La méthodologie employée dans cette analyse consiste à créer une grille de 1 km x 1 km et à associer les musées à cette grille en fonction de leurs coordonnées géographiques. Les données sont ensuite regroupées par cellule de la grille, permettant de compter le nombre de musées dans chaque zone. Cette approche permet une visualisation efficace de la répartition géographique des musées. La requête SQL générée utilise les fonctions spatiales de DuckDB pour réaliser ces opérations.


Les résultats de l'analyse sont ensuite visualisés à l'aide de Plotly Express, qui permet de créer une carte choroplèthe représentant la densité de musées dans chaque zone de la grille. Cette visualisation interactive offre une vue d'ensemble claire et détaillée de la répartition des musées en France.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets/liste-et-localisation-des-musees-de-france/exports/parquet?lang=fr&timezone=Europe%2FBerlin", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH 
-- Créer une grille de 1km x 1km et associer les musées à cette grille
grid AS (
  SELECT 
    CAST(longitude AS DOUBLE) AS lon, 
    CAST(latitude AS DOUBLE) AS lat,
    FLOOR(lon * 100) / 100 AS grid_lon,
    FLOOR(lat * 100) / 100 AS grid_lat
  FROM loaded_dataset
  WHERE longitude IS NOT NULL AND latitude IS NOT NULL
)
-- Regrouper les musées par cellule de la grille
SELECT 
  grid_lon, 
  grid_lat, 
  COUNT(*) AS count_museums,
  ST_AsText(ST_Envelope(ST_Union_Agg(ST_GeomFromText('POINT(' || lon || ' ' || lat || ')')))) AS geom
FROM grid
GROUP BY grid_lon, grid_lat
ORDER BY grid_lon, grid_lat """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 1036 lignes


,grid_lon,grid_lat,count_museums,geom
0,-61.65,15.85,1,POINT (-61.646677 15.850088)
1,-61.54,16.23,3,"POLYGON ((-61.537737 16.236416, -61.535945 16...."
2,-61.30,15.87,1,POINT (-61.298004 15.876666)
3,-61.18,14.74,1,POINT (-61.175994 14.743103)
4,-61.08,14.64,1,POINT (-61.070648 14.649226)


## 📈 Visualisation

La bibliothèque principale utilisée est Plotly Express, qui est adaptée pour créer des visualisations interactives de données géospatiales. Le choix d'une carte de densité (density mapbox) permet de représenter efficacement la répartition des musées sur une carte du monde. Cela permet une exploration interactive des données.

In [4]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.express as px

# Préparation
gdf = df.rename(columns={"grid_lon": "lon", "grid_lat": "lat"})
gdf["taille"] = gdf["count_museums"].astype(str)

# Carte choroplèthe
dataviz = px.density_mapbox(
    gdf,
    lat="lat",
    lon="lon",
    z="count_museums",
    radius=7,
    zoom=1.5,
    mapbox_style="carto-positron",
    color_continuous_scale="Viridis",
    hover_data={
        "lon": True,
        "lat": True,
        "count_museums": True,
        "taille": False
    },
    labels={"count_museums": "Nb musées", "lon": "Longitude", "lat": "Latitude"},
)

dataviz.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    font=dict(family="Roboto"),
    coloraxis_colorbar_title="Nb musées"
)
dataviz

C:\Users\ami\AppData\Local\Temp\ipykernel_1716\17635644.py:11: DeprecationWarning: *density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  dataviz = px.density_mapbox(


---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250805_063037.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250805_063037.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250805_063037.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_20250805_063037.png


--> Image Plotly sauvegardée avec succès.
